In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym
import numpy as np

from IPython import get_ipython
import random
import matplotlib.pyplot as plt
from IPython import display
from tqdm.notebook import tqdm
from dynamic_env import TaskEnv_drift


In [ ]:

class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, 64),  # 输入维度是 state_dim
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_dim),  # 输出维度是 action_dim
            nn.Softmax(dim=-1)
        )

    def forward(self, state):
        return self.fc(state)

# 定义 MAML 算法
class MAML:
    def __init__(self, state_dim, action_dim, lr_inner=0.01, lr_outer=0.001):
        self.policy = PolicyNetwork(state_dim, action_dim)
        self.lr_inner = lr_inner  # 内循环学习率
        self.lr_outer = lr_outer  # 外循环学习率
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr_outer)

    def adapt(self, task, num_steps=1):
        """
        内循环：在单个任务上快速适应
        """
        fast_weights = list(self.policy.parameters())  # 获取所有参数的副本
        for step in range(num_steps):
            states, actions, rewards = self.sample_trajectory(task)
            loss = self.compute_loss(states, actions, rewards)
            grads = torch.autograd.grad(loss, fast_weights, create_graph=True)
            fast_weights = [w - self.lr_inner * g for w, g in zip(fast_weights, grads)]
        return fast_weights

    def meta_train(self, tasks: list[TaskEnv_drift], num_iterations=20):
        """
        外循环：在多个任务上优化初始参数
        """
        for iteration in range(num_iterations):
            self.optimizer.zero_grad()
            meta_loss = 0
            for task in tasks:
                fast_weights = self.adapt(task)
                states, actions, rewards = self.sample_trajectory(task, fast_weights)
                loss = self.compute_loss(states, actions, rewards, fast_weights)
                meta_loss += loss
            meta_loss /= len(tasks)
            meta_loss.backward()
            self.optimizer.step()
            print(f"Iteration {iteration}, Meta Loss: {meta_loss.item()}")

    def compute_loss(self, states, actions, rewards, params=None):
        """
        计算策略梯度损失
        """
        if params is None:
            probs = self.policy(states)
        else:
            probs = self.forward_with_params(states, params)
        log_probs = torch.log(probs.gather(1, actions.unsqueeze(1)))
        loss = -(log_probs * rewards).mean()
        return loss

    def forward_with_params(self, states, params):
        x = states
        param_iter = iter(params)  # 创建一个参数迭代器
        for layer in self.policy.fc:
            if isinstance(layer, nn.Linear):
                weight = next(param_iter)  # 获取权重
                bias = next(param_iter)    # 获取偏置
                x = torch.nn.functional.linear(x, weight, bias)
            else:
                x = layer(x)
        return x

    def sample_trajectory(self, task: TaskEnv_drift, params=None):
        """
        在任务中采样一条轨迹
        """
        states, actions, rewards = [], [], []
        state = task.reset()
        done = False
        while not done:
            # 将状态转换为 one-hot 编码
            state_idx = task.states.index(state)  # 获取状态的索引
            state_onehot = np.zeros(len(task.states))  # 创建 one-hot 向量
            state_onehot[state_idx] = 1
            state_tensor = torch.FloatTensor(state_onehot)  # 转换为 PyTorch 张量

            # 选择动作
            if params is None:
                action_probs = self.policy(state_tensor)
            else:
                action_probs = self.forward_with_params(state_tensor, params)
            action = torch.multinomial(action_probs, 1).item()

            # 执行动作
            next_state, reward, done, _ = task.step(action)

            # 存储轨迹数据
            states.append(state_tensor)
            actions.append(action)
            rewards.append(reward)
            state = next_state

        # 将轨迹数据转换为 PyTorch 张量
        states = torch.stack(states)
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        return states, actions, rewards



In [22]:
# 主程序
if __name__ == "__main__":
    # 定义任务
    tasks = []
    for _ in range(4):
        task = TaskEnv_drift()
        if np.random.rand() < 0.5:  # 50% 的概率添加 drift
            task.set_flag()
            task.drift(add_actions=0, add_states=0)  # 只有transition metrix变化
        tasks.append(task)

    print(tasks[0])

    # 初始化 MAML
    state_dim = len(tasks[0].states)  # 状态空间大小
    action_dim = len(tasks[0].motions)  # 动作空间大小
    print(state_dim,action_dim)



  

drift happen
drift happen
drift happen
drift happen
<TaskEnv_drift instance>
5 7


/Users/veetin/Desktop/drifttest_pa/main/dynamic_env.py:47: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  frequencies[label][action] = ast.literal_eval(frequencies[label][action]) #判断需要计算的内容是不是合法的Python类型，如果是则执行，否则就报错
/Users/veetin/Desktop/dr

In [25]:
    maml = MAML(state_dim, action_dim)
    # 元训练
    maml.meta_train(tasks, num_iterations=10)

Iteration 0, Meta Loss: -4.0154032707214355
Iteration 1, Meta Loss: -3.3095688819885254
Iteration 2, Meta Loss: 0.49231788516044617
Iteration 3, Meta Loss: -2.5166015625
Iteration 4, Meta Loss: -3.198366641998291
Iteration 5, Meta Loss: -6.794022560119629
Iteration 6, Meta Loss: -2.3016016483306885
Iteration 7, Meta Loss: -2.749467372894287
Iteration 8, Meta Loss: -1.114458441734314
Iteration 9, Meta Loss: -3.8135173320770264


In [17]:
test_task = TaskEnv_drift()
test_task.set_flag()
test_task.drift(add_actions=0, add_states=0)  # 添加 2 个新动作和 2 个新状态
fast_weights = maml.adapt(test_task)
states, actions, rewards = maml.sample_trajectory(test_task, fast_weights)
test_loss = maml.compute_loss(states, actions, rewards, fast_weights)
print(f"Test Loss: {test_loss.item()}")

drift happen
Test Loss: 1.8209065198898315


/Users/veetin/Desktop/drifttest_pa/main/dynamic_env.py:47: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  frequencies[label][action] = ast.literal_eval(frequencies[label][action]) #判断需要计算的内容是不是合法的Python类型，如果是则执行，否则就报错
